In [1]:
import tensorflow as tf
from tensorflow.contrib.model_pruning.python import pruning
from tensorflow.contrib.model_pruning.python.layers import layers
import keras
from utils import *

Using TensorFlow backend.


In [2]:
epochs = 50
epochs_prune = 5
batch_size = 250 
model_path_unpruned = "Model_Saves/Unpruned{}.ckpt"
model_path_pruned = "Model_Saves/Pruned{}.ckpt"
NUM_CLASS = 10

In [3]:
#Load Dataset
X_train0, y_train = prepare_dataset(data_dir, 'train')
X_test0, y_test = prepare_dataset(data_dir, 'test')
t = int(time.time())

#Normalizing
mean = np.mean(X_train0,axis=(0,1,2,3))
std = np.std(X_train0,axis=(0,1,2,3))
np.save('mean',mean)
np.save('std',std)
X_train = z_normalization(X_train0, mean, std)
X_test = z_normalization(X_test0, mean, std)

#Labels to binary
y_train_binary = keras.utils.to_categorical(y_train,num_classes)
y_test_binary = keras.utils.to_categorical(y_test,num_classes)

batches = int(len(X_train) / batch_size)
batches_test = int(len(X_test) / batch_size)
print(batches)

Loading numpy
Loading numpy
200


In [4]:
def sample_batch(dataset, labels, batch_size):
    N = dataset.shape[0]
    indices = np.random.randint(N, size=batch_size)
    x_epoch = dataset[indices]
    y_epoch = labels[indices]
    return x_epoch, y_epoch

def test_batch(dataset, labels, batch_size):
    for index, offset in enumerate(range(0, dataset.shape[0], batch_size)):
        x_epoch, y_epoch = np.array(dataset[offset: offset + batch_size,:]), np.array(labels[offset: offset +  batch_size])
    return x_epoch, y_epoch

In [5]:
tf.reset_default_graph()

image = tf.placeholder(name='images', dtype=tf.float32, shape=[None, 32, 32, 3])
label = tf.placeholder(name='fine_labels', dtype=tf.int32, shape=[None, 10])

_=image
_ = layers.masked_conv2d(_, 96, (3, 3), 1, 'SAME')
_ = tf.layers.batch_normalization(_, name='norm1-1')
_ = layers.masked_conv2d(_, 96, (3, 3), 1, 'SAME')
_ = tf.layers.batch_normalization(_, name='norm1-2')
_ = tf.layers.max_pooling2d(_, (3, 3), 2, 'SAME',name='pool1')
_ = layers.masked_conv2d(_, 192, (3, 3), 1, 'SAME')
_ = tf.layers.batch_normalization(_, name='norm2-1')
_ = layers.masked_conv2d(_, 192, (3, 3), 1, 'SAME')
_ = tf.layers.batch_normalization(_, name='norm2-2')
_ = tf.layers.max_pooling2d(_, (3, 3), 2, 'SAME', name='pool2')
_ = layers.masked_conv2d(_, 192, (3, 3), 1, 'VALID')
_ = tf.layers.batch_normalization(_, name='norm3')
_ = layers.masked_conv2d(_, 192, (1, 1), 1)
_ = tf.layers.batch_normalization(_, name='norm4')
_ = layers.masked_conv2d(_, 10, (1, 1), 1)
_ = tf.layers.batch_normalization(_, name='norm5')
_ = tf.layers.average_pooling2d(_, (6,6), 1, name='avg_pool')
y = _
logits = tf.reshape(y,[tf.shape(y)[0],10])


# Create global step variable (needed for pruning)
global_step = tf.train.get_or_create_global_step()
reset_global_step_op = tf.assign(global_step, 0)

# Loss function

loss = tf.losses.softmax_cross_entropy(label, logits)

# Training op, the global step is critical here, make sure it matches the one used in pruning later
# running this operation increments the global_step
train_op = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(loss, global_step=global_step)

# Accuracy ops
prediction = tf.cast(tf.argmax(logits, axis=1), tf.int32)
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(label, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
accuracy_5 = tf.reduce_mean(tf.cast(tf.nn.in_top_k(predictions=logits, targets=tf.argmax(label, 1), k=5), tf.float32))


In [6]:
# Get, Print, and Edit Pruning Hyperparameters
def set_prune_params(s):
    pruning_hparams = pruning.get_pruning_hparams()
    print("Pruning Hyperparameters:", pruning_hparams)

    # Change hyperparameters to meet our needs
    pruning_hparams.begin_pruning_step = 0
    pruning_hparams.end_pruning_step = 250
    pruning_hparams.pruning_frequency = 1
    pruning_hparams.sparsity_function_end_step = 250
    pruning_hparams.target_sparsity = s

    # Create a pruning object using the pruning specification, sparsity seems to have priority over the hparam
    p = pruning.Pruning(pruning_hparams, global_step=global_step) #sparsity=.5)
    prune_op = p.conditional_mask_update_op()
    return prune_op

# Create a saver for writing training checkpoints.
saver = tf.train.Saver()



In [7]:
with tf.Session() as sess:

    # Uncomment the following if you don't have a trained model yet
    sess.run(tf.global_variables_initializer())

    # Train the model before pruning (optional)
    for epoch in range(epochs):
        for batch in range(batches):
            batch_xs, batch_ys = sample_batch(X_train, y_train_binary, batch_size)
            sess.run(train_op, feed_dict={image: batch_xs, label: batch_ys})

        # Calculate Test Accuracy every 5 epochs
        if (epoch+1) % 5 == 0:
            acc_print = 0
            acc_print_5 = 0
            for index, offset in enumerate(range(0, X_test.shape[0], batch_size)):
                batch_xt = np.array(X_test[offset: offset + batch_size,:]) 
                batch_yt = np.array(y_test_binary[offset: offset +  batch_size])
                acc_print += sess.run(accuracy, feed_dict={image: batch_xt, label: batch_yt})
                acc_print_5 += sess.run(accuracy_5, feed_dict={image: batch_xt, label: batch_yt})
            print("Un-pruned model epoch %d test accuracy %g" % (epoch+1, acc_print/batches_test))
            print("Un-pruned model epoch %d test top5-accuracy %g" % (epoch+1, acc_print_5/batches_test))
            
            # Saves the model before pruning
            saver.save(sess, model_path_unpruned.format(epoch))
        print(epoch+1)
            
        
    print("Top-1 Pre-Pruning accuracy:", acc_print/batches_test)
    print("Top-5 Pre-Pruning accuracy:", acc_print_5/batches_test)
    print("Sparsity of layers (should be 0)", sess.run(tf.contrib.model_pruning.get_weight_sparsity()))


1
2
3
4
Un-pruned model epoch 5 test accuracy 0.7963
Un-pruned model epoch 5 test top5-accuracy 0.9906
5
6
7
8
9
Un-pruned model epoch 10 test accuracy 0.8186
Un-pruned model epoch 10 test top5-accuracy 0.992
10
11
12
13
14
Un-pruned model epoch 15 test accuracy 0.832
Un-pruned model epoch 15 test top5-accuracy 0.9932
15
16
17
18
19
Un-pruned model epoch 20 test accuracy 0.8245
Un-pruned model epoch 20 test top5-accuracy 0.9891
20
21
22
23
24
Un-pruned model epoch 25 test accuracy 0.8239
Un-pruned model epoch 25 test top5-accuracy 0.9933
25
26
27
28
29
Un-pruned model epoch 30 test accuracy 0.8304
Un-pruned model epoch 30 test top5-accuracy 0.9898
30
31
32
33
34
Un-pruned model epoch 35 test accuracy 0.8354
Un-pruned model epoch 35 test top5-accuracy 0.9911
35
36
37
38
39
Un-pruned model epoch 40 test accuracy 0.84
Un-pruned model epoch 40 test top5-accuracy 0.9908
40
41
42
43
44
Un-pruned model epoch 45 test accuracy 0.8323
Un-pruned model epoch 45 test top5-accuracy 0.9926
45
46
47
4

In [7]:
sparsity = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]

In [8]:
for s in sparsity:
    prune_op = set_prune_params(s)
    print('Target sparsity is {}'.format(s))
    
    with tf.Session() as sess:
        # Resets the session and restores the saved model
        sess.run(tf.global_variables_initializer())
        saver.restore(sess, model_path_unpruned.format(44))

        # Reset the global step counter and begin pruning
        sess.run(reset_global_step_op)
        for epoch in range(epochs_prune):
            for batch in range(batches):
                batch_xs, batch_ys = sample_batch(X_train, y_train_binary, batch_size)
                # Prune and retrain
                sess.run(prune_op)
                sess.run(train_op, feed_dict={image: batch_xs, label: batch_ys})

    #         # Calculate Test Accuracy every 10 epochs
    #         if epoch % 1 == 0:
    #             acc_print = 0
    #             acc_print_5 = 0
    #             for batch in range(batches):
    #                 batch_xt, batch_yt = sample_batch(X_test, y_test_binary, batch_size)
    #                 acc_print += sess.run(accuracy, feed_dict={image: batch_xt, label: batch_yt})
            # Calculate Test Accuracy every 10 epochs
            if (epoch+1) % 1 == 0:
                acc_print = 0
                acc_print_5 = 0
                for index, offset in enumerate(range(0, X_test.shape[0], batch_size)):
                    batch_xt = np.array(X_test[offset: offset + batch_size,:]) 
                    batch_yt = np.array(y_test_binary[offset: offset +  batch_size])
    #             for batch in range(batches_test):
    #                 batch_xt, batch_yt = test_batch(X_test, y_test_binary, batch_size)
                    acc_print += sess.run(accuracy, feed_dict={image: batch_xt, label: batch_yt})
                    acc_print_5 += sess.run(accuracy_5, feed_dict={image: batch_xt, label: batch_yt})

                print("Pruned model step %d test accuracy %g" % (epoch, acc_print/batches_test))
                print("Pruned model step %d test top5-accuracy %g" % (epoch, acc_print_5/batches_test))
                print("Weight sparsities:", sess.run(tf.contrib.model_pruning.get_weight_sparsity()))
            print(epoch) 

               # acc_print = sess.run(accuracy, feed_dict={image: mnist.test.images, label: mnist.test.labels})

        # Saves the model after pruning
        saver.save(sess, model_path_pruned.format(s))

        # Print final accuracy
        #acc_print = sess.run(accuracy, feed_dict={image: mnist.test.images, label: mnist.test.labels})
        print("Final accuracy:", acc_print/batches_test)
        print("Final sparsity by layer", sess.run(tf.contrib.model_pruning.get_weight_sparsity()))

Pruning Hyperparameters: [('begin_pruning_step', 0), ('block_height', 1), ('block_pooling_function', 'AVG'), ('block_width', 1), ('end_pruning_step', -1), ('initial_sparsity', 0), ('name', 'model_pruning'), ('nbins', 256), ('pruning_frequency', 10), ('sparsity_function_begin_step', 0), ('sparsity_function_end_step', 100), ('sparsity_function_exponent', 3), ('target_sparsity', 0.5), ('threshold_decay', 0.9), ('use_tpu', False), ('weight_sparsity_map', [''])]
INFO:tensorflow:Updating masks.
Target sparsity is 0.1
INFO:tensorflow:Restoring parameters from Model_Saves/Unpruned44.ckpt
Pruned model step 0 test accuracy 0.8367
Pruned model step 0 test top5-accuracy 0.9912
Weight sparsities: [0.09915123, 0.09343654, 0.087981045, 0.09233037, 0.09408155, 0.09423828, 0.096875]
0
Pruned model step 1 test accuracy 0.8413
Pruned model step 1 test top5-accuracy 0.9902
Weight sparsities: [0.099922836, 0.09938031, 0.097668305, 0.09804808, 0.09902163, 0.09901258, 0.09791667]
1
Pruned model step 2 test a

Final accuracy: 0.8578999951481819
Final sparsity by layer [0.4992284, 0.49240452, 0.49746817, 0.49004146, 0.49528295, 0.49666342, 0.49791667]
Pruning Hyperparameters: [('begin_pruning_step', 0), ('block_height', 1), ('block_pooling_function', 'AVG'), ('block_width', 1), ('end_pruning_step', -1), ('initial_sparsity', 0), ('name', 'model_pruning'), ('nbins', 256), ('pruning_frequency', 10), ('sparsity_function_begin_step', 0), ('sparsity_function_end_step', 100), ('sparsity_function_exponent', 3), ('target_sparsity', 0.5), ('threshold_decay', 0.9), ('use_tpu', False), ('weight_sparsity_map', [''])]
INFO:tensorflow:Updating masks.
Target sparsity is 0.6
INFO:tensorflow:Restoring parameters from Model_Saves/Unpruned44.ckpt
Pruned model step 0 test accuracy 0.8366
Pruned model step 0 test top5-accuracy 0.9906
Weight sparsities: [0.58989197, 0.58575666, 0.5856542, 0.58273655, 0.58281493, 0.588406, 0.5859375]
0
Pruned model step 1 test accuracy 0.8486
Pruned model step 1 test top5-accuracy 0

Final accuracy: 0.7702999994158745
Final sparsity by layer [0.9506173, 0.95024353, 0.9499843, 0.9497884, 0.9498246, 0.9496528, 0.94947916]
Pruning Hyperparameters: [('begin_pruning_step', 0), ('block_height', 1), ('block_pooling_function', 'AVG'), ('block_width', 1), ('end_pruning_step', -1), ('initial_sparsity', 0), ('name', 'model_pruning'), ('nbins', 256), ('pruning_frequency', 10), ('sparsity_function_begin_step', 0), ('sparsity_function_end_step', 100), ('sparsity_function_exponent', 3), ('target_sparsity', 0.5), ('threshold_decay', 0.9), ('use_tpu', False), ('weight_sparsity_map', [''])]
INFO:tensorflow:Updating masks.
Target sparsity is 0.99
INFO:tensorflow:Restoring parameters from Model_Saves/Unpruned44.ckpt
Pruned model step 0 test accuracy 0.3692
Pruned model step 0 test top5-accuracy 0.8765
Weight sparsities: [0.97492284, 0.97480226, 0.9752544, 0.97541416, 0.9749982, 0.97488064, 0.9703125]
0
Pruned model step 1 test accuracy 0.3903
Pruned model step 1 test top5-accuracy 0.8

In [13]:
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file
print_tensors_in_checkpoint_file('Model_Saves/Unpruned44.ckpt', 
                                 all_tensors=False, all_tensor_names=False, tensor_name='')

Conv/biases (DT_FLOAT) [96]
Conv/biases/Adam (DT_FLOAT) [96]
Conv/biases/Adam_1 (DT_FLOAT) [96]
Conv/mask (DT_FLOAT) [3,3,3,96]
Conv/threshold (DT_FLOAT) []
Conv/weights (DT_FLOAT) [3,3,3,96]
Conv/weights/Adam (DT_FLOAT) [3,3,3,96]
Conv/weights/Adam_1 (DT_FLOAT) [3,3,3,96]
Conv_1/biases (DT_FLOAT) [96]
Conv_1/biases/Adam (DT_FLOAT) [96]
Conv_1/biases/Adam_1 (DT_FLOAT) [96]
Conv_1/mask (DT_FLOAT) [3,3,96,96]
Conv_1/threshold (DT_FLOAT) []
Conv_1/weights (DT_FLOAT) [3,3,96,96]
Conv_1/weights/Adam (DT_FLOAT) [3,3,96,96]
Conv_1/weights/Adam_1 (DT_FLOAT) [3,3,96,96]
Conv_2/biases (DT_FLOAT) [192]
Conv_2/biases/Adam (DT_FLOAT) [192]
Conv_2/biases/Adam_1 (DT_FLOAT) [192]
Conv_2/mask (DT_FLOAT) [3,3,96,192]
Conv_2/threshold (DT_FLOAT) []
Conv_2/weights (DT_FLOAT) [3,3,96,192]
Conv_2/weights/Adam (DT_FLOAT) [3,3,96,192]
Conv_2/weights/Adam_1 (DT_FLOAT) [3,3,96,192]
Conv_3/biases (DT_FLOAT) [192]
Conv_3/biases/Adam (DT_FLOAT) [192]
Conv_3/biases/Adam_1 (DT_FLOAT) [192]
Conv_3/mask (DT_FLOAT) [3,

In [21]:
print_tensors_in_checkpoint_file('Model_Saves/Unpruned44.ckpt', 
                                 all_tensors=False, all_tensor_names=False, tensor_name='Conv/weights')

tensor_name:  Conv/weights
[[[[ 0.04379602 -0.05505071 -0.08778767 ...  0.03066595 -0.04357576
    -0.05353526]
   [ 0.0070363   0.04169472 -0.0729975  ...  0.08931225 -0.06831206
     0.10006074]
   [ 0.12743428  0.07768662  0.0874206  ... -0.02181112 -0.04949465
     0.09114436]]

  [[-0.0539235  -0.03097104  0.01627925 ...  0.05563014 -0.09813255
     0.1472321 ]
   [ 0.00965081  0.02887369  0.01724323 ... -0.02030833 -0.11718856
     0.07390618]
   [ 0.08303585 -0.03235126  0.12467189 ... -0.1112793   0.02824809
     0.08702562]]

  [[-0.15823641 -0.06146544  0.00619495 ...  0.031168    0.04163508
     0.13076901]
   [-0.11911135  0.11352054 -0.05747633 ...  0.02983151 -0.07704147
     0.10038877]
   [-0.01146747  0.01727173  0.14272916 ...  0.06176776  0.0394776
     0.03551508]]]


 [[[ 0.04224705  0.04160998 -0.02847889 ... -0.11748441 -0.06114323
     0.15397218]
   [ 0.06881077  0.05409041 -0.01212053 ... -0.13167563 -0.11558327
    -0.02307469]
   [-0.05883789 -0.02132129  0.

In [33]:
with tf.Session() as sess1:
    sess1.run("Conv/weights")

In [25]:
vars = tf.trainable_variables()
print(vars) #some infos about variables...
# with tf.Session() as sess1:
#     vars_vals = sess1.run(vars)
# for var, val in zip(vars, vars_vals):
#     print("var: {}, value: {}".format(var.name, val)) 

[<tf.Variable 'Conv/weights:0' shape=(3, 3, 3, 96) dtype=float32_ref>, <tf.Variable 'Conv/biases:0' shape=(96,) dtype=float32_ref>, <tf.Variable 'norm1-1/gamma:0' shape=(96,) dtype=float32_ref>, <tf.Variable 'norm1-1/beta:0' shape=(96,) dtype=float32_ref>, <tf.Variable 'Conv_1/weights:0' shape=(3, 3, 96, 96) dtype=float32_ref>, <tf.Variable 'Conv_1/biases:0' shape=(96,) dtype=float32_ref>, <tf.Variable 'norm1-2/gamma:0' shape=(96,) dtype=float32_ref>, <tf.Variable 'norm1-2/beta:0' shape=(96,) dtype=float32_ref>, <tf.Variable 'Conv_2/weights:0' shape=(3, 3, 96, 192) dtype=float32_ref>, <tf.Variable 'Conv_2/biases:0' shape=(192,) dtype=float32_ref>, <tf.Variable 'norm2-1/gamma:0' shape=(192,) dtype=float32_ref>, <tf.Variable 'norm2-1/beta:0' shape=(192,) dtype=float32_ref>, <tf.Variable 'Conv_3/weights:0' shape=(3, 3, 192, 192) dtype=float32_ref>, <tf.Variable 'Conv_3/biases:0' shape=(192,) dtype=float32_ref>, <tf.Variable 'norm2-2/gamma:0' shape=(192,) dtype=float32_ref>, <tf.Variable '

In [65]:
 with tf.Session() as sess:
        # Resets the session and restores the saved model
        sess.run(tf.global_variables_initializer())
        #saver.restore(sess, model_path_unpruned.format(44))
        saver.restore(sess, model_path_pruned.format(0.9))
        # get the graph
        g = tf.get_default_graph()
        w1 = g.get_tensor_by_name('Conv_2/threshold:0')
        x = sess.run(w1)

INFO:tensorflow:Restoring parameters from Model_Saves/Pruned0.9.ckpt


In [66]:
x

0.16020398

In [40]:
total_parameters = 0
for variable in tf.trainable_variables():
    shape = variable.get_shape()
    variable_parametes = 1
    for dim in shape:
        variable_parametes *= dim.value
    total_parameters += variable_parametes
print("Total training params: %.5fM" % (total_parameters / 1e6))

Total training params: 0.95667M


In [47]:
with tf.Session() as sess:
    # Resets the session and restores the saved model
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, model_path_pruned.format(0.9))
    tvars = tf.trainable_variables()
    tvars_vals = sess.run(tvars)

    for var, val in zip(tvars, tvars_vals):
        print(var.name, val)

INFO:tensorflow:Restoring parameters from Model_Saves/Pruned0.9.ckpt
Conv/weights:0 [[[[ 0.04386877 -0.05370045 -0.08818778 ...  0.03163777 -0.03516616
    -0.05234836]
   [ 0.00367678  0.03967545 -0.07397655 ...  0.08231409 -0.06926255
     0.09879471]
   [ 0.12032126  0.06689382  0.08355619 ... -0.02391103 -0.04284841
     0.09426026]]

  [[-0.0479385  -0.02921759  0.01969598 ...  0.05949609 -0.08868785
     0.24975243]
   [ 0.00721393  0.03141775  0.01831823 ... -0.01895187 -0.12521599
     0.07258015]
   [ 0.07973339 -0.03227412  0.14145286 ... -0.10893171  0.03382431
     0.09231466]]

  [[-0.14586031 -0.06017137  0.00895014 ...  0.03246674  0.05327276
     0.13533467]
   [-0.1040989   0.0871282  -0.05544589 ...  0.03138801 -0.06712543
     0.11880873]
   [-0.01569531  0.01624121  0.19445866 ...  0.06390522  0.04735205
     0.03824019]]]


 [[[ 0.0377103   0.03618093 -0.02557194 ... -0.10170904 -0.05498583
     0.1387983 ]
   [ 0.06660452  0.05048921 -0.01265561 ... -0.14740188 -0

In [ ]:
with tf.Session() as sess:
    tvars = tf.trainable_variables()
    for var in tvars:
        if var.name.endswith('weights:0'):
            val = sess.run(var)
            print(var.name, val)

In [ ]:
epochs = 3
batch_size = 512 # Entire training set
model_path_unpruned = "Model_Saves/Unpruned.ckpt"
model_path_pruned = "Model_Saves/Pruned.ckpt"
NUM_CLASS = 10
tf.reset_default_graph()

# Import dataset
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
batches = int(len(mnist.train.images) / batch_size)

# Define Placeholders
# image = tf.placeholder(tf.float32, [None, 28,28])
# label = tf.placeholder(tf.float32, [None, 10])

image = tf.placeholder(name='images', dtype=tf.float32, shape=[None, 28, 28, 1])
label = tf.placeholder(name='fine_labels', dtype=tf.int32, shape=[None,10])

# # Define the model
# layer1 = layers.masked_fully_connected(image, 300)
# layer2 = layers.masked_fully_connected(layer1, 300)
# logits = layers.masked_fully_connected(layer2, 10)

# Define the model
# layer1 = layers.masked_conv2d(image, 300, kernel_size=2)
# layer2 = layers.masked_conv2d(layer1, 300, kernel_size=2)
# logits = layers.masked_fully_connected(layer2, 10)

_=image
_ = layers.masked_conv2d(_, 96, (3, 3), 1, 'SAME')
_ = tf.layers.batch_normalization(_, name='norm1-1')
_ = layers.masked_conv2d(_, 96, (3, 3), 1, 'SAME')
_ = tf.layers.batch_normalization(_, name='norm1-2')
_ = tf.layers.max_pooling2d(_, (3, 3), 2, 'SAME',name='pool1')
_ = layers.masked_conv2d(_, 192, (3, 3), 1, 'SAME')
_ = tf.layers.batch_normalization(_, name='norm2-1')
_ = layers.masked_conv2d(_, 192, (3, 3), 1, 'SAME')
_ = tf.layers.batch_normalization(_, name='norm2-2')
_ = tf.layers.max_pooling2d(_, (3, 3), 2, 'SAME', name='pool2')
_ = layers.masked_conv2d(_, 192, (3, 3), 1, 'VALID')
_ = tf.layers.batch_normalization(_, name='norm3')
_ = layers.masked_conv2d(_, 192, (1, 1), 1)
_ = tf.layers.batch_normalization(_, name='norm4')
_ = layers.masked_conv2d(_, 10, (1, 1), 1)
_ = tf.layers.batch_normalization(_, name='norm5')
_ = tf.layers.average_pooling2d(_, (5,5), 1, name='avg_pool')
y = _
logits = tf.reshape(y,[tf.shape(y)[0],10])


# Create global step variable (needed for pruning)
global_step = tf.train.get_or_create_global_step()
reset_global_step_op = tf.assign(global_step, 0)

# Loss function
#loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=label))
#loss = tf.losses.softmax_cross_entropy(tf.one_hot(label, NUM_CLASS), logits)
loss = tf.losses.softmax_cross_entropy(label, logits)

# Training op, the global step is critical here, make sure it matches the one used in pruning later
# running this operation increments the global_step
train_op = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss, global_step=global_step)

# Accuracy ops
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(label, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))



In [ ]:
# Get, Print, and Edit Pruning Hyperparameters
pruning_hparams = pruning.get_pruning_hparams()
print("Pruning Hyperparameters:", pruning_hparams)

# Change hyperparameters to meet our needs
pruning_hparams.begin_pruning_step = 0
pruning_hparams.end_pruning_step = 250
pruning_hparams.pruning_frequency = 1
pruning_hparams.sparsity_function_end_step = 250
pruning_hparams.target_sparsity = .5

# Create a pruning object using the pruning specification, sparsity seems to have priority over the hparam
p = pruning.Pruning(pruning_hparams, global_step=global_step) #sparsity=.5)
prune_op = p.conditional_mask_update_op()

# Create a saver for writing training checkpoints.
saver = tf.train.Saver()



In [ ]:
with tf.Session() as sess:

    # Uncomment the following if you don't have a trained model yet
    sess.run(tf.global_variables_initializer())

    # Train the model before pruning (optional)
    for epoch in range(epochs):
        for batch in range(batches):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            #print(batch_xs.shape)
            batch_xs = batch_xs.reshape(-1,28,28,1)
            sess.run(train_op, feed_dict={image: batch_xs, label: batch_ys})

        # Calculate Test Accuracy every 10 epochs
        if epoch % 1 == 0:
            acc_print = 0
            for batch in range(batches):
                batch_xt, batch_yt = mnist.test.next_batch(batch_size)
                #print(batch_xt.shape)
                batch_xt = batch_xt.reshape(-1,28,28,1)
                acc_print += sess.run(accuracy, feed_dict={image: batch_xt, label: batch_yt})
            print("Un-pruned model step %d test accuracy %g" % (epoch, acc_print/batches))
        print(epoch)
    
    # Saves the model before pruning
    saver.save(sess, model_path_unpruned)
    
    acc_print = sess.run(accuracy, feed_dict={image: mnist.test.images.reshape(-1,28,28,1), label: mnist.test.labels})
    print("Pre-Pruning accuracy:", acc_print)
    print("Sparsity of layers (should be 0)", sess.run(tf.contrib.model_pruning.get_weight_sparsity()))

    

    # Resets the session and restores the saved model
    sess.run(tf.initialize_all_variables())
    saver.restore(sess, model_path_unpruned)

    # Reset the global step counter and begin pruning
    sess.run(reset_global_step_op)
    for epoch in range(epochs):
        for batch in range(batches):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            batch_xs = batch_xs.reshape(-1,28,28,1)
            # Prune and retrain
            sess.run(prune_op)
            sess.run(train_op, feed_dict={image: batch_xs, label: batch_ys})

        # Calculate Test Accuracy every 10 epochs
        if epoch % 1 == 0:
            acc_print = 0
            for batch in range(batches):
                batch_xt, batch_yt = mnist.test.next_batch(batch_size)
                #print(batch_xt.shape)
                batch_xt = batch_xt.reshape(-1,28,28,1)
                acc_print += sess.run(accuracy, feed_dict={image: batch_xt, label: batch_yt})
            
            print("Pruned model step %d test accuracy %g" % (epoch, acc_print/batches))
            print("Weight sparsities:", sess.run(tf.contrib.model_pruning.get_weight_sparsity()))
        print(epoch) 
        
           # acc_print = sess.run(accuracy, feed_dict={image: mnist.test.images, label: mnist.test.labels})

    # Saves the model after pruning
    saver.save(sess, model_path_pruned)

    # Print final accuracy
    #acc_print = sess.run(accuracy, feed_dict={image: mnist.test.images, label: mnist.test.labels})
    print("Final accuracy:", acc_print)
    print("Final sparsity by layer (should be 0)", sess.run(tf.contrib.model_pruning.get_weight_sparsity()))

In [ ]:
mnist

In [ ]:
batches